# Description

For this exercise, you will need to create 5 processes that communicate data between each other in some manner.  As with other exercises in this course, there will be serial and non-concurrent ways to achieve this; your job is to use actual interprocess communication and/or shared memory for the task.

Remember that every process has a process ID assigned by the operating system, and queryable using `os.getpid()`.  Using that capability, we perform a simple computation that will give a different result every time we launch processes and depend on each process contributing.

* Process one will receive an integer, K, as input from the parent, and calculate $N_1 = (K^2 + pid)\ (mod\ 36721)$.
* Process two will receive the number calculated from process one, $N_1$, and calculate $N_2 = (N_1^2 + pid)\ (mod\ 36721)$.
* ... etc through process five ...
* Process five will calculate $(N_4^2 + pid)\ (mod\ 36721)$ and communicate that result with the parent.

Obviously, for this example, a process' PID is merely a stand-in for data it would contain/calculate in a real-world parallel computation.

# Setup



In [1]:
from multiprocessing import Process, Queue, Pipe, Pool
from os import getpid
from time import sleep

def worker():
    print(getpid())
    while True:
        sleep(0.1)
    
processes = [Process(target=worker) for _ in range(5)]
result = 0

def fill_result(K):
    global result
    for p in processes:
        if not p.is_alive():
            p.start()
    # ... do something else ...
    result = 12345

# Solution

In [2]:
pipes = [Pipe() for _ in range(6)]

def worker(number):
    _, recipient = pipes[number]
    sender, _ = pipes[number+1]
    while True:
        data = recipient.recv()
        output = (data**2 + getpid()) % 36721
        sender.send(output)

processes = [Process(target=worker, args=(i,)) for i in range(5)]

def fill_result(K):
    global result
    for p in processes:
        if not p.is_alive():
            p.start()
    sender, _ = pipes[0]
    sender.send(K)
    _, recipient = pipes[5]
    result = recipient.recv()

# Test Cases

In [3]:
def test_processes():
    assert len(processes) == 5
    for p in processes:
        assert isinstance(p, Process)
        
test_processes()

In [4]:
def test_result():
    global result
    from random import randint
    
    sample_K = (randint(1000, 10_000) for _ in range(5))

    for K in sample_K:
        fill_result(K)
        N1 = (K**2 + processes[0].pid) % 36721
        N2 = (N1**2 + processes[1].pid) % 36721
        N3 = (N2**2 + processes[2].pid) % 36721
        N4 = (N3**2 + processes[3].pid) % 36721
        N5 = (N4**2 + processes[4].pid) % 36721
        assert N5 == result, f"K={K}, correct={N5}, calculated={result}"
    
test_result()